In [6]:
import numpy as np
from itertools import combinations
import random

# Constants for the 3D simulation
BOX_WIDTH = 50
BOX_HEIGHT = 50
BOX_LENGTH = 50
PARTICLE_RADII = {'A': 3, 'B': 3, 'C': 3}  # Adjusted for species-specific radii
INITIAL_SPEEDS = {'A': 4, 'B': 4, 'C': 4}  # Adjusted for species-specific speeds
SIMULATION_SETS = 5  # Number of simulations per particle number set
MAX_PARTICLES = 7  # Maximum number of each particle in the system

# Particle class adjusted for 3D
class Particle:
    def __init__(self, species, x, y, z, vx, vy, vz):
        self.species = species
        self.x = x
        self.y = y
        self.z = z
        self.vx = vx
        self.vy = vy
        self.vz = vz
        self.radius = PARTICLE_RADII[species]

# Functions update_positions and handle_collisions remain unchanged
# Function to update positions in 3D
def update_positions(particles):
    for particle in particles:
        particle.x += particle.vx
        particle.y += particle.vy
        particle.z += particle.vz
        # Handle wall collisions in 3D
        if particle.x > BOX_WIDTH - particle.radius or particle.x < particle.radius:
            particle.vx *= -1
        if particle.y > BOX_HEIGHT - particle.radius or particle.y < particle.radius:
            particle.vy *= -1
        if particle.z > BOX_LENGTH - particle.radius or particle.z < particle.radius:
            particle.vz *= -1

def handle_collisions(particles):
    for p1, p2 in combinations(particles, 2):
        dx = p1.x - p2.x
        dy = p1.y - p2.y
        dz = p1.z - p2.z
        distance = np.sqrt(dx**2 + dy**2 + dz**2)

        if distance < (p1.radius + p2.radius):
            n = np.array([dx, dy, dz]) / distance
            v1i = np.array([p1.vx, p1.vy, p1.vz])
            v2i = np.array([p2.vx, p2.vy, p2.vz])
            
            v1n = np.dot(v1i, n) * n
            v2n = np.dot(v2i, n) * n
            
            v1t = v1i - v1n
            v2t = v2i - v2n
            
            v1nf = v2n
            v2nf = v1n
            
            # Update velocities for inelastic collision
            p1.vx, p1.vy, p1.vz = (v1nf + v1t) * 0.9
            p2.vx, p2.vy, p2.vz = (v2nf + v2t) * 0.9

            # Change species after collision based on rules
            if {p1.species, p2.species} == {'A', 'B'}:
                p1.species = 'C'
                p2.species = 'C'
            elif {p1.species, p2.species} == {'A', 'C'}:
                p1.species = 'B'
                p2.species = 'B'
            elif {p1.species, p2.species} == {'B', 'C'}:
                p1.species = 'A'
                p2.species = 'A'
            # If same species collide, no change in species


# Function to run a single 3D simulation
def run_simulation(num_particles):
    particles = []
    for species in ['A', 'B', 'C']:
        for _ in range(num_particles):
            angle_theta = random.uniform(0, 2 * np.pi)
            angle_phi = random.uniform(0, np.pi)
            speed = INITIAL_SPEEDS[species]
            vx = speed * np.sin(angle_phi) * np.cos(angle_theta)
            vy = speed * np.sin(angle_phi) * np.sin(angle_theta)
            vz = speed * np.cos(angle_phi)
            particles.append(Particle(species,
                                      random.uniform(PARTICLE_RADII[species], BOX_WIDTH - PARTICLE_RADII[species]),
                                      random.uniform(PARTICLE_RADII[species], BOX_HEIGHT - PARTICLE_RADII[species]),
                                      random.uniform(PARTICLE_RADII[species], BOX_LENGTH - PARTICLE_RADII[species]),
                                      vx, vy, vz))
    step = 0
    while step < 1000000:  # No step limit
        update_positions(particles)
        handle_collisions(particles)
        remaining_species = set(particle.species for particle in particles)
        if len(remaining_species) == 1:
            break
        step += 1
    return remaining_species, step

for num_particles in range(1, MAX_PARTICLES + 1):
    print(f"Running simulations with {num_particles} of each particle type.")
    steps_recorded = []
    for set_num in range(SIMULATION_SETS):
        outcome, steps = run_simulation(num_particles)
        print(f"Set {set_num + 1}: Remaining species {outcome}, steps taken {steps}.")
        steps_recorded.append(steps)
    print(f"Average steps for {num_particles} particles: {np.mean(steps_recorded):.2f}")
    # Increase the number of each type of particle for the next round of simulations, up to a maximum
    NUM_PARTICLES = min(num_particles + 1, MAX_PARTICLES)

Running simulations with 1 of each particle type.
Set 1: Remaining species {'B'}, steps taken 124.
Set 2: Remaining species {'A'}, steps taken 65.
Set 3: Remaining species {'B'}, steps taken 111.
Set 4: Remaining species {'B'}, steps taken 53.
Set 5: Remaining species {'C'}, steps taken 134.
Average steps for 1 particles: 97.40
Running simulations with 2 of each particle type.
Set 1: Remaining species {'C'}, steps taken 2027.
Set 2: Remaining species {'A'}, steps taken 4353.
Set 3: Remaining species {'C'}, steps taken 235.
Set 4: Remaining species {'A', 'B', 'C'}, steps taken 1000000.
Set 5: Remaining species {'B'}, steps taken 128.
Average steps for 2 particles: 201348.60
Running simulations with 3 of each particle type.
Set 1: Remaining species {'A', 'B'}, steps taken 1000000.
Set 2: Remaining species {'A', 'B', 'C'}, steps taken 1000000.
Set 3: Remaining species {'B', 'C'}, steps taken 1000000.
Set 4: Remaining species {'A', 'B', 'C'}, steps taken 1000000.
Set 5: Remaining species {

KeyboardInterrupt: 